From paper:
https://github.com/maitreyeeT/Seq2SeqDialStruct/blob/431419204a4156d633d0d744657fc0cbeef08ce6/Seq2SeqVAE_dialogueStructuring/vae_LSTM.py#L39

Look for tutorials on VAE. preferably pytorch? (or not, i don't think i used it yet)

In [ ]:
import keras
# from Seq2SeqVAE_dialogueStructuring.variational_layer import VariationalLayer
l = keras.layers
m = keras.models
K = keras.backend
opt = keras.optimizers
losses = keras.losses
cbk = keras.callbacks
reg = keras.regularizers


class LSTMVAutoencoder:

    def __init__(self, lr, window_size, sequence_size, dropout=0.):
        self.lr = lr
        self.window_size = window_size
        self.sequences_length = sequence_size
       # self.input_convolution_size = 3
       # self.conv_2d_filters = []  # Num filter of the convolution layers
       # self.conv_2d_kernels = []  # Kernel sizes of the convolution layers

        self.dropout = dropout
        self.regul = 1e-4
        self.latent_dim = 256
        self.build()

    def get_attention(self,name, output_size, dropout=0., axis=-2):
        model_att = m.Sequential(name='attention_' + str(name))  # (?, n, k, x)
        model_att.add(l.Dense(1, activation=K.exp))  # (?, n, k, 1)
        model_att.add(l.Lambda(lambda x: x / (K.sum(x, axis=axis, keepdims=True) + \
                                              K.epsilon()),
                               name=name + 'attention'))  # (?, n, k, 1)

        m_att = m.Sequential()
        m_att.add(l.Lambda(lambda x: K.sum(x * model_att(x), axis=axis)))
        m_att.add(l.Dense(output_size, activation='linear'))
        return m_att

    def make_networks(self):
        attention = self.get_attention('xylo',10)
        #from w2vec_utils import train_word2vec, data_transform
        #data_embed = data_transform()
        #w2vmodel, embed_wts = train_word2vec(data_embed)
        encoder = m.Sequential(name='encoder')
        #encoder.add(l.Embedding(output_dim=300, weights=[embed_wts],input_dim=embed_wts.shape[0],
        #                        input_shape=(self.sequences_length,2)))
        #encoder.add(l.Lambda(lambda x: K.mean(x, axis=-2)))
        encoder.add(l.LSTM(10, return_sequences='True'))
        encoder.add(l.LSTM(10, return_sequences='True'))

        encoder.add(attention)
        #encoder.add(l.Dense(5, activity_regularizer=reg.l2(1e-3)))

        self.encoder = encoder

        decoder = m.Sequential(name='decoder')
        decoder.add(l.RepeatVector(self.sequences_length))
        decoder.add(l.LSTM(10, return_sequences=True))
        decoder.add(l.LSTM(self.window_size, activation='sigmoid'
                           , return_sequences=True))

        self.decoder = decoder

    def build(self):

        seq_input = l.Input(shape=(self.sequences_length, self.window_size),
                            dtype='float32')  # (?, n, k)

        self.make_networks()

        latent_in = self.encoder(seq_input)
    #     latent_in = VariationalLayer(10, kl_gain=1e-3)(latent_in) <<<< variational layer. commented since i don't yet hve the code for this

        decoder_out = self.decoder(latent_in)

        model = m.Model(inputs=[seq_input], outputs=[decoder_out])

        model_latent = m.Model(inputs=[seq_input], outputs=[latent_in])

        model.compile(opt.Adam(self.lr), loss=[lambda x, y: losses.binary_crossentropy(x, y)])

        model_latent.compile(opt.RMSprop(1e-3), loss='binary_crossentropy')
        self.model = model
        self.model_latent = model_latent

    def fit(self, X, epochs, callbacks=None, verbose=2):

        self.model.fit(x=[X],
                       y=[X],
                       batch_size=30,
                       epochs=epochs,
                       validation_split=.2,
                       callbacks=callbacks,
                       verbose=verbose)

## LSTM Autoencoders

https://machinelearningmastery.com/lstm-autoencoders/

In [12]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils.vis_utils import plot_model



In [1]:
# For these demonstrations, we will use a dataset of one sample of nine time steps and one feature:
# [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# We can start-off by defining the sequence and reshaping it into the preferred shape of [samples, timesteps, features].

# define input sequence
sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 1))

# Next, we can define the encoder-decoder LSTM architecture that expects input sequences with nine time steps
#and one feature and outputs a sequence with nine time steps and one feature.

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

# Next, we can fit the model on our contrived dataset.

# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)
# plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])


2022-05-02 10:42:52.024640: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 10:42:52.745231: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


[0.10486767 0.19934608 0.29777828 0.39852583 0.50044525 0.6015635
 0.7009781  0.7998287  0.89919174]


In [11]:
# define input sequence
seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))
# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
# plot_model(model, show_shapes=True, to_file='predict_lstm_autoencoder.png')

# fit model
model.fit(seq_in, seq_out, epochs=300, verbose=0)
# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat[0,:,0])


[0.16533749 0.2886588  0.40279546 0.50950813 0.6105695  0.7070858
 0.7999728  0.89002454]


In [40]:
# # lstm autoencoder reconstruct and predict sequence
# from numpy import array
# import keras
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import RepeatVector
# from keras.layers import TimeDistributed

# m = keras.models
# l = keras.layers
# # Finally, we can create a composite LSTM Autoencoder that has a single encoder and two decoders,
# # one for reconstruction and one for prediction.




# # define model
# model = Sequential()
# model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
# model.add(RepeatVector(n_out))
# model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(TimeDistributed(Dense(1)))
# model.compile(optimizer='adam', loss='mse')
# # plot_model(model, show_shapes=True, to_file='predict_lstm_autoencoder.png')


# ##########

# # define input sequence
# seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# # reshape input into [samples, timesteps, features]
# n_in = len(seq_in)
# seq_in = seq_in.reshape((1, n_in, 1))
# # prepare output sequence
# seq_out = seq_in[:, 1:, :]
# n_out = n_in - 1

# # define encoder
# visible = Input(shape=(n_in,1))
# encoder = LSTM(100, activation='relu')(visible)
# print(encoder)

# # define reconstruct decoder
# decoder1 = RepeatVector(n_in)(encoder)
# # decoder1_l1 = LSTM(100, activation='relu', return_sequences=True)(decoder1)
# # decoder1_l2 = LSTM(100, activation='relu', return_sequences=True)(decoder1_l1)
# # decoder1_o = TimeDistributed(Dense(1))(decoder1_l2) #layer wrapper
# decoder1 = LSTM(100, activation='relu', return_sequences=True)(decoder1)
# decoder1 = TimeDistributed(Dense(1))(decoder1) #layer wrapper
# print('decoder1: '+ str(decoder1))
# print('\n')

# # define predict decoder
# decoder2 = RepeatVector(n_out)(encoder)
# decoder2_l1 = LSTM(100, activation='relu', return_sequences=True)(decoder2)
# decoder2_l2 = LSTM(100, activation='relu', return_sequences=True)(decoder2_l1)
# decoder2_o = TimeDistributed(Dense(1))(decoder2_l2) #layer wrapper
# print('decoder2: '+ str(decoder2_o))
# print('\n')

# # tie it together
# model = Model(inputs=visible, outputs=[decoder1, decoder2])
# model.compile(optimizer='adam', loss='mse')

# # fit model
# model.fit(seq_in, [seq_in,seq_out], epochs=300, verbose=0)

# # demonstrate prediction
# yhat = model.predict(seq_in, verbose=0)
# print('yhat: ' + str(yhat))

In [51]:
# from src/ngrams import add_special_tok
import pandas as pd
import sys
from ngrams import add_special_tok

df = pd.read_csv('./data_TM2/processed/processed_utterances_sentence_DA_labeling.csv', index_col=0)
df.head()

add_special_tok(df)

KeyboardInterrupt: 

In [39]:

# lstm autoencoder reconstruct and predict sequence
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# define input sequence
seq_in = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))
# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define encoder
visible = Input(shape=(n_in,1))
encoder = LSTM(100, activation='relu')(visible)

# define reconstruct decoder
decoder1 = RepeatVector(n_in)(encoder)
decoder1 = LSTM(100, activation='relu', return_sequences=True)(decoder1)
decoder1 = TimeDistributed(Dense(1))(decoder1)

# define predict decoder
decoder2 = RepeatVector(n_out)(encoder)
decoder2 = LSTM(100, activation='relu', return_sequences=True)(decoder2)
decoder2 = TimeDistributed(Dense(1))(decoder2)

# tie it together
model = Model(inputs=visible, outputs=[decoder1, decoder2])
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq_in, [seq_in,seq_out], epochs=300, verbose=0)

# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat)

[array([[[0.11162275],
        [0.20765445],
        [0.30347523],
        [0.39957055],
        [0.49649748],
        [0.59487337],
        [0.6952509 ],
        [0.79859203],
        [0.9059592 ]]], dtype=float32), array([[[0.16432385],
        [0.28716987],
        [0.40201792],
        [0.50994134],
        [0.61183226],
        [0.7084466 ],
        [0.80043477],
        [0.888363  ]]], dtype=float32)]
